<a href="https://colab.research.google.com/github/hseyeon1006/ESAA_YB/blob/main/%EB%AF%B8%EB%8B%88%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/학회/데이터/wine_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/학회/데이터/wine_test.csv')

In [4]:
# id값은 필요 없으므로 제거한다.
train=train.drop(['index'],axis=1)
test=test.drop(['index'],axis=1)

In [5]:
# 각 컬럼별로 scale이 다르므로, 표준화 시켜준다. 이 때, type은 수치형이 아니므로 잠깐 제외하고 표준화를 진행한다.
standardtrain=train.drop(['type','quality'],axis=1)
standardtrain=(standardtrain-standardtrain.mean())/standardtrain.std()
standardtrain=pd.merge(standardtrain,train['type'],left_index=True,right_index=True)

In [6]:
# test 데이터도 표준화 진행
standardtest=test.drop(['type'],axis=1)
standardtest=(standardtest-standardtest.mean())/standardtest.std()
test=pd.merge(standardtest,test['type'],left_index=True,right_index=True)

In [7]:
# type의 white와 red를 0,1로 바꾸기
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
standardtrain['type'] = enc.transform(standardtrain['type'])
test['type'] = enc.transform(test['type'])

In [8]:
X_train= standardtrain
y_train=train['quality']
X_test = test

In [14]:
#랜덤포레스트
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
params={'n_estimators':[50,100,150,200,400,500],'max_features':[0.2,0.4,0.6,0.8,1.0]}
gridsearch=GridSearchCV(model,params,cv=5)
gridsearch.fit(X_train, y_train)
gridsearch.best_params_

{'max_features': 0.2, 'n_estimators': 200}

In [11]:
model = RandomForestClassifier(max_features=0.2,n_estimators=500)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=12, max_features=1.0, min_samples_leaf=8,
                       min_samples_split=8, n_estimators=300)

In [ ]:
#결정 트리
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

model = DecisionTreeClassifier()
params = {'max_depth':[6,8,10,12,16,20],'min_samples_split':[16,24]}
gridsearch=GridSearchCV(model,params,cv=5)
gridsearch.fit(X_train, y_train)
gridsearch.best_params_

{'max_depth': 8, 'min_samples_split': 16}

In [ ]:
model = DecisionTreeClassifier(max_depth=8,min_samples_split=16)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=8, min_samples_split=16)

In [ ]:
#GBM
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

model = GradientBoostingClassifier()
params={'n_estimators':[100,500],'learning_rate':[0.05,0.01]}
gridsearch=GridSearchCV(model,params,cv=5,verbose=1)
gridsearch.fit(X_train, y_train)
gridsearch.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


{'learning_rate': 0.05, 'n_estimators': 500}

In [ ]:
gridsearch.best_score_

0.6137869137232194

In [12]:
predict=model.predict(X_test)

In [13]:
submission = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/학회/데이터/wine_sample_submission.csv')
submission['quality'] = predict
submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/학회/미니 프로젝트/baseline.csv', index=False)